In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

FILE_PATH = "/content/drive/MyDrive/genesExpression/RABC13_Level3.txt"  # <-- change this

df = pd.read_csv(FILE_PATH, sep="\t")
print(df.shape)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(19921, 100)


,symbol,GSM1381406,GSM1381407,GSM1381408,GSM1381430,GSM1381431,GSM1381432,GSM1381433,GSM1381524,GSM1381525,...,GSM1382141,GSM1382142,GSM1382143,GSM1382144,GSM1382145,GSM1382146,GSM1382147,GSM1382148,GSM1382149,GSM1382150
0,A1BG,5.034736,5.153410,4.771349,5.489528,4.913313,4.938768,5.051148,5.952392,6.285110,...,6.155497,6.130924,5.382765,5.785689,6.058182,5.783485,6.024936,5.708449,5.829147,5.930300
1,A1CF,3.580576,4.112098,4.133986,4.167102,3.732367,3.768247,4.379803,4.356622,4.453563,...,4.203970,4.258301,4.181870,4.213459,4.166540,4.122978,4.474931,4.077597,4.103557,4.239628
2,A2BP1,2.817091,2.817612,3.048173,3.198213,2.944027,3.059729,2.869777,3.126752,3.136976,...,3.100237,2.988933,2.975656,3.886131,3.133297,3.045693,3.025082,3.007035,3.162358,3.071836
3,A2LD1,4.053511,4.305152,4.077550,4.042980,3.934547,4.055266,3.759129,4.341665,4.666297,...,4.232469,4.259745,3.880452,4.016038,4.091444,4.148738,4.416004,4.098573,4.411988,4.347975
4,A2M,8.787091,9.036652,9.014434,8.947403,8.691395,8.679810,8.834901,5.024601,5.072084,...,4.659070,5.100233,4.721538,4.840841,4.933884,4.634589,4.783980,4.863360,4.596491,4.830779


In [ ]:
import numpy as np

# Try common gene column names
possible_gene_cols = ["symbol", "gene", "Gene", "GENE", "gene_symbol"]
gene_col = next((c for c in possible_gene_cols if c in df.columns), df.columns[0])

# Drop obvious non-expression columns (optional)
drop_cols = [c for c in ["index", "Index", "ID", "id"] if c in df.columns]

work = df.drop(columns=drop_cols, errors="ignore").copy()

# Gene column
work[gene_col] = work[gene_col].astype(str)

# Keep numeric expression columns only (these are your samples)
numeric_cols = []
for c in work.columns:
    if c == gene_col:
        continue
    s = pd.to_numeric(work[c], errors="coerce")
    if s.notna().mean() > 0.8:  # at least 80% numeric
        numeric_cols.append(c)

print("Gene column:", gene_col)
print("Number of sample columns detected:", len(numeric_cols))
print("First sample columns:", numeric_cols[:5])


Gene column: symbol
Number of sample columns detected: 99
First sample columns: ['GSM1381406', 'GSM1381407', 'GSM1381408', 'GSM1381430', 'GSM1381431']


3) Select one sample (patient) + compute up/down signature

This is your “patient state” built from a cohort-style matrix.

B) Build a “patient signature” from expression (simple, works for demos)

Pick one sample column (a GSM column) and compute z-scores across samples per gene:

In [ ]:
expr = work[[gene_col] + numeric_cols].copy()
expr = expr.dropna(subset=[gene_col]).drop_duplicates(subset=[gene_col])

# Convert to numeric
mat = expr[numeric_cols].apply(pd.to_numeric, errors="coerce")
mat.index = expr[gene_col].values
#standarize format : DGIdb expects gene symbols like STAT3, not STAT3.1 or extra spaces.
mat.index = (
    mat.index.astype(str)
    .str.upper()
    .str.replace(r"\.\d+$", "", regex=True)
    .str.strip()
)

# Pick a patient sample (change this to any column you want)
patient_sample = numeric_cols[0]
patient_vec = mat[patient_sample]

# Z-score patient vs cohort for each gene
mu = mat.mean(axis=1)
sd = mat.std(axis=1).replace(0, np.nan)
z = ((patient_vec - mu) / sd).dropna()

up_genes = z.sort_values(ascending=False).head(200).index.tolist()
down_genes = z.sort_values().head(200).index.tolist()

print("Patient sample:", patient_sample)
print("Top up genes:", up_genes[:10])
print("Top down genes:", down_genes[:10])


Patient sample: GSM1381406
Top up genes: ['LBP', 'SEMA3A', 'BAI3', 'RSPO2', 'IP6K3', 'WDR69', 'PAQR5', 'LRRC2', 'CCBE1', 'TRIM9']
Top down genes: ['SUB1', 'RPL29', 'MATR3', 'HNRNPUL1', 'C2ORF24', 'EIF1', 'SHOC2', 'RPL30', 'KIF24', 'CNN2']


C) DGIdb: map patient genes → candidate drugs

Use dgipy, then inspect available query functions (so you don’t get stuck on function names):

In [ ]:
!pip -q uninstall -y gql dgipy graphql-core
!pip -q install dgipy==0.1.0 "gql[requests]==3.5.0"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 107.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 165.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 25.1 MB/s eta 0:00:00


In [ ]:

import dgipy
[f for f in dir(dgipy) if f.startswith("get_")]


['get_all_genes',
 'get_categories',
 'get_drug_applications',
 'get_drugs',
 'get_genes',
 'get_interactions',
 'get_sources']

In [ ]:
import inspect

print("get_interactions signature:", inspect.signature(dgipy.get_interactions))
help(dgipy.get_interactions)


get_interactions signature: (terms: list, search: str = 'genes', immunotherapy: bool | None = None, antineoplastic: bool | None = None, source: str | None = None, pmid: int | None = None, interaction_type: str | None = None, approved: str | None = None, api_url: str | None = None) -> dict
Help on function get_interactions in module dgipy.dgidb:

get_interactions(terms: list, search: str = 'genes', immunotherapy: bool | None = None, antineoplastic: bool | None = None, source: str | None = None, pmid: int | None = None, interaction_type: str | None = None, approved: str | None = None, api_url: str | None = None) -> dict
    Perform an interaction look up for drugs or genes of interest

    :param terms: drugs or genes for interaction look up
    :param search: interaction search type. valid types are "drugs" or "genes"
    :param immunotherapy: filter option for results that are used in immunotherapy
    :param antineoplastic: filter option for results that are part of antineoplastic reg

In [ ]:
import dgipy, gql
print("dgipy:", getattr(dgipy, "__version__", "unknown"))
print("gql:", gql.__version__)


dgipy: unknown
gql: 3.5.0


creates interactions (the DGIdb results dataframe)

In [ ]:
import pandas as pd

def chunks(lst, n=50):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

parts = []
for glist in chunks(up_genes[:200], 50):
    res = dgipy.get_interactions(terms=glist, search="genes")
    parts.append(pd.DataFrame(res))

interactions = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()
print("Rows:", len(interactions))
print("Columns:", interactions.columns.tolist())
interactions.head()


Rows: 866
Columns: ['gene_name', 'gene_concept_id', 'gene_long_name', 'drug_name', 'drug_concept_id', 'drug_approved', 'interaction_score', 'interaction_attributes', 'interaction_sources', 'interaction_pmids']


,gene_name,gene_concept_id,gene_long_name,drug_name,drug_concept_id,drug_approved,interaction_score,interaction_attributes,interaction_sources,interaction_pmids
0,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,CLOBAZAM,rxcui:21241,True,0.085767,"{'Interaction Context': None, 'Endogenous Drug...","[TEND, ChEMBL, TdgClinicalTrial]",[]
1,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,CLORAZEPATE DIPOTASSIUM,rxcui:2607,True,0.039310,"{'Interaction Context': None, 'Endogenous Drug...",[ChEMBL],[]
2,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,GANAXOLONE,rxcui:2604689,True,0.069885,"{'Interaction Context': None, 'Endogenous Drug...","[ChEMBL, TdgClinicalTrial]",[]
3,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,CLONAZEPAM,rxcui:2598,True,0.075475,"{'Interaction Context': None, 'Endogenous Drug...","[ChEMBL, TEND, TdgClinicalTrial]",[]
4,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,BROMAZEPAM,rxcui:1749,True,0.059901,"{'Interaction Context': None, 'Endogenous Drug...","[TdgClinicalTrial, TEND]",[]


since we are going to merge DGIdb output with the IC50 output, so we must standardize.

In [ ]:
# ===== Normalize DGIdb interactions dataframe schema (run AFTER interactions is created) =====
print("Before normalization:", interactions.columns.tolist())

# Try to standardize likely column names coming from DGIdb
rename_map = {}
for c in interactions.columns:
    lc = c.lower()

    if lc in ["gene_name", "gene", "genesymbol", "gene_symbol", "geneSymbol".lower()]:
        rename_map[c] = "gene"
    if lc in ["drug_name", "drug", "drugname", "drug_name_normalized"]:
        rename_map[c] = "drug"
    if lc in ["interaction_types", "interaction_type", "interactiontype", "types"]:
        rename_map[c] = "interaction_types"
    if lc in ["sources", "source"]:
        rename_map[c] = "sources"

interactions = interactions.rename(columns=rename_map)

# Ensure required columns exist
for col in ["gene", "drug", "interaction_types", "sources"]:
    if col not in interactions.columns:
        interactions[col] = None

# Clean values
interactions["gene"] = interactions["gene"].astype(str).str.upper().str.strip()
interactions["drug"] = interactions["drug"].astype(str).str.strip()

print("After normalization:", interactions.columns.tolist())
print("Interactions shape:", interactions.shape)
interactions.head()


Before normalization: ['gene_name', 'gene_concept_id', 'gene_long_name', 'drug_name', 'drug_concept_id', 'drug_approved', 'interaction_score', 'interaction_attributes', 'interaction_sources', 'interaction_pmids']
After normalization: ['gene', 'gene_concept_id', 'gene_long_name', 'drug', 'drug_concept_id', 'drug_approved', 'interaction_score', 'interaction_attributes', 'interaction_sources', 'interaction_pmids', 'interaction_types', 'sources']
Interactions shape: (866, 12)


,gene,gene_concept_id,gene_long_name,drug,drug_concept_id,drug_approved,interaction_score,interaction_attributes,interaction_sources,interaction_pmids,interaction_types,sources
0,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,CLOBAZAM,rxcui:21241,True,0.085767,"{'Interaction Context': None, 'Endogenous Drug...","[TEND, ChEMBL, TdgClinicalTrial]",[],None,None
1,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,CLORAZEPATE DIPOTASSIUM,rxcui:2607,True,0.039310,"{'Interaction Context': None, 'Endogenous Drug...",[ChEMBL],[],None,None
2,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,GANAXOLONE,rxcui:2604689,True,0.069885,"{'Interaction Context': None, 'Endogenous Drug...","[ChEMBL, TdgClinicalTrial]",[],None,None
3,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,CLONAZEPAM,rxcui:2598,True,0.075475,"{'Interaction Context': None, 'Endogenous Drug...","[ChEMBL, TEND, TdgClinicalTrial]",[],None,None
4,GABRA4,hgnc:4078,gamma-aminobutyric acid type A receptor subuni...,BROMAZEPAM,rxcui:1749,True,0.059901,"{'Interaction Context': None, 'Endogenous Drug...","[TdgClinicalTrial, TEND]",[],None,None


In [ ]:
# Try to find drug/gene columns in the returned dataframe
possible_gene_cols = ["gene_name", "gene", "geneSymbol", "gene_symbol"]
possible_drug_cols = ["drug_name", "drug", "drugName", "drug_name_normalized"]

gene_c = next((c for c in possible_gene_cols if c in interactions.columns), None)
drug_c = next((c for c in possible_drug_cols if c in interactions.columns), None)

print("Detected gene column:", gene_c)
print("Detected drug column:", drug_c)

if gene_c and drug_c:
    interactions[gene_c] = interactions[gene_c].astype(str).str.upper()
    up_set = set([g.upper() for g in up_genes])

    # keep only interactions that match your up genes
    inter_up = interactions[interactions[gene_c].isin(up_set)].copy()

    # score drugs by number of unique up genes they hit
    drug_scores = (
    interactions.dropna(subset=["drug"])
    .groupby("drug")["gene"]
    .nunique()
    .sort_values(ascending=False)
    .reset_index(name="num_patient_genes_hit")
    )
# 20 kefaya 3shan akeed msh h model l adwya aktar mn kedaaaaaaaaaaa
    TOP_N = 20
    top_drugs = drug_scores.head(TOP_N)["drug"].tolist()

    print("Top drugs:", top_drugs[:10])
    drug_scores.head(20)

else:
    print("Could not detect gene/drug columns. Check interactions.columns and set them manually.")


Detected gene column: gene
Detected drug column: drug
Top drugs: ['ENFLURANE', 'HALOTHANE', 'ISOFLURANE', 'DESFLURANE', 'SEVOFLURANE', 'METHOXYFLURANE', 'LORAZEPAM', 'TOPIRAMATE', 'PROPOFOL', 'REMIMAZOLAM BESYLATE']


recommendation list: drugs most connected to the patient’s dysregulated genes (based on DGIdb).

In [ ]:
drug_scores.head(20)


,drug,num_patient_genes_hit
0,ENFLURANE,6
1,HALOTHANE,5
2,ISOFLURANE,5
3,DESFLURANE,5
4,SEVOFLURANE,5
5,METHOXYFLURANE,4
6,LORAZEPAM,4
7,TOPIRAMATE,4
8,PROPOFOL,4
9,REMIMAZOLAM BESYLATE,3


) Build the patient feature vector (from your matrix)

In [ ]:
import numpy as np

# Patient feature vector: use z-scores (recommended so scale matches)
patient_features = z.copy()  # index = gene symbol, value = z-score
patient_features.name = "patient_z"

# Keep only finite
patient_features = patient_features.replace([np.inf, -np.inf], np.nan).dropna()
print("Patient features:", patient_features.shape)


Patient features: (19921,)


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
# ==== PRISM inputs (upload these to /content or set paths to where you saved them) ====
PRISM_DOSE_RESPONSE_PATH = "/content/drive/MyDrive/prisma ds/secondary-screen-dose-response-curve-parameters.csv"
PRISM_CELL_LINE_INFO_PATH = "/content/drive/MyDrive/prisma ds/secondary-screen-cell-line-info.csv"
PRISM_TREATMENT_INFO_PATH = "/content/drive/MyDrive/prisma ds/secondary-screen-replicate-collapsed-treatment-info.csv"



print("PRISM dose-response file:", PRISM_DOSE_RESPONSE_PATH)
print("PRISM cell-line info file:", PRISM_CELL_LINE_INFO_PATH)
print("PRISM treatment info file:", PRISM_TREATMENT_INFO_PATH)


PRISM dose-response file: /content/drive/MyDrive/prisma ds/secondary-screen-dose-response-curve-parameters.csv
PRISM cell-line info file: /content/drive/MyDrive/prisma ds/secondary-screen-cell-line-info.csv
PRISM treatment info file: /content/drive/MyDrive/prisma ds/secondary-screen-replicate-collapsed-treatment-info.csv


In [ ]:
# ===== Expression file(s) =====
# If this is YOUR patient expression (single patient), keep it here:
PATIENT_EXPR_PATH = "/content/drive/MyDrive/genesExpression/RABC13_Level3.txt"

# If you later download cell-line expression for PRISM/DepMap, put it here:
# CELL_LINE_EXPR_PATH = "/content/Cell_line_RMA_proc_basalExp.txt"
CELL_LINE_EXPR_PATH = None

In [ ]:
# ==== Load PRISM secondary screen dose-response parameters ====
prism = pd.read_csv(PRISM_DOSE_RESPONSE_PATH)

# clean column names
prism.columns = [c.strip() for c in prism.columns]

print("PRISM shape:", prism.shape)
print("PRISM columns (first 40):", prism.columns[:40].tolist())

# --- Helper: pick the first existing column from a list ---
def pick_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

# ---- Detect key columns (these names vary by PRISM release) ----
cell_col = pick_col(prism, ["depmap_id", "DepMap_ID", "cell_line", "cell_line_name", "cell_id", "broad_id"])
drug_col = pick_col(prism, ["name", "compound_name", "drug", "pert_iname", "broad_compound_id", "compound_id"])

# target candidates (PRISM usually provides AUC / EC50 / IC50 / logIC50-like columns depending on file)
log10_ic50_col = pick_col(prism, ["log10_ic50", "logIC50", "log_ic50", "log10IC50"])
ic50_uM_col     = pick_col(prism, ["ic50", "IC50", "ic50_um", "IC50_uM", "ec50", "EC50"])
auc_col         = pick_col(prism, ["auc", "AUC", "aac", "AAC"])

print("Detected cell_col:", cell_col)
print("Detected drug_col:", drug_col)
print("Detected log10_ic50_col:", log10_ic50_col)
print("Detected ic50_uM_col:", ic50_uM_col)
print("Detected auc_col:", auc_col)

if cell_col is None or drug_col is None:
    raise ValueError("Could not detect cell line column or drug column. Show me prism.columns and I will map them exactly.")


/tmp/ipython-input-2439368592.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  prism = pd.read_csv(PRISM_DOSE_RESPONSE_PATH)


PRISM shape: (701004, 20)
PRISM columns (first 40): ['broad_id', 'depmap_id', 'ccle_name', 'screen_id', 'upper_limit', 'lower_limit', 'slope', 'r2', 'auc', 'ec50', 'ic50', 'name', 'moa', 'target', 'disease.area', 'indication', 'smiles', 'phase', 'passed_str_profiling', 'row_name']
Detected cell_col: depmap_id
Detected drug_col: name
Detected log10_ic50_col: None
Detected ic50_uM_col: ic50
Detected auc_col: auc


Build the training target table (IC50 if exists, else AUC)

In [ ]:
# Build response table (cell line × drug × target)
train_resp = prism[["depmap_id", "name", "ic50", "auc"]].copy()
train_resp.columns = ["sample_id", "drug", "ic50_uM", "auc"]

# Clean ids
train_resp["sample_id"] = train_resp["sample_id"].astype(str).str.strip()
train_resp["drug"] = train_resp["drug"].astype(str).str.strip()

# Convert IC50 to numeric
train_resp["ic50_uM"] = pd.to_numeric(train_resp["ic50_uM"], errors="coerce")

# Convert IC50 (uM) -> ln(M)
# IMPORTANT: if ic50_uM <= 0, log is invalid, so we drop those
train_resp = train_resp[train_resp["ic50_uM"].notna() & (train_resp["ic50_uM"] > 0)].copy()
train_resp["ln_ic50"] = np.log(train_resp["ic50_uM"] * 1e-6)

print("train_resp shape:", train_resp.shape)
print(train_resp[["sample_id", "drug", "ic50_uM", "ln_ic50", "auc"]].head())


train_resp shape: (361898, 5)
      sample_id         drug    ic50_uM    ln_ic50       auc
533  ACH-000569   epinastine  32.524180 -10.333527  0.979333
820  ACH-000879  floxuridine   0.501776 -14.505112  0.726373
821  ACH-000320  floxuridine   0.007879 -18.659009  0.336463
822  ACH-001145  floxuridine   0.041552 -16.996323  0.485618
823  ACH-000873  floxuridine   0.097745 -16.140901  0.548483


In [ ]:
CELL_EXPR_PATH = "/content/drive/MyDrive/prisma ds/CCLE_expression.csv"  # change this

expr = pd.read_csv(CELL_EXPR_PATH)
print("expr shape:", expr.shape)
print(expr.columns[:15])
print(expr.head(3))


expr shape: (1406, 19222)
Index(['Unnamed: 0', 'TSPAN6 (7105)', 'TNMD (64102)', 'DPM1 (8813)',
       'SCYL3 (57147)', 'C1orf112 (55732)', 'FGR (2268)', 'CFH (3075)',
       'FUCA2 (2519)', 'GCLC (2729)', 'NFYA (4800)', 'STPG1 (90529)',
       'NIPAL3 (57185)', 'LAS1L (81887)', 'ENPP4 (22875)'],
      dtype='object')
   Unnamed: 0  TSPAN6 (7105)  TNMD (64102)  DPM1 (8813)  SCYL3 (57147)  \
0  ACH-001113       4.331992      0.000000     7.364397       2.792855   
1  ACH-001289       4.566815      0.584963     7.106537       2.543496   
2  ACH-001339       3.150560      0.000000     7.379032       2.333424   

   C1orf112 (55732)  FGR (2268)  CFH (3075)  FUCA2 (2519)  GCLC (2729)  ...  \
0          4.470537    0.028569    1.226509      3.042644     6.499686  ...   
1          3.504620    0.000000    0.189034      3.813525     4.221104  ...   
2          4.227279    0.056584    1.310340      6.687061     3.682573  ...   

   H3C2 (8358)  H3C3 (8352)  AC098582.1 (8916)  DUS4L-BCAP29 (11525

1) Clean + prepare the expression matrix (features)

You need depmap_id to merge with PRISM.

You want gene symbols only so they match DGIdb later.

You force numeric values so ML won’t break.

In [ ]:
import re

# 1) Rename the cell line id column
expr = expr.rename(columns={"Unnamed: 0": "depmap_id"})

# 2) Clean gene column names: "TSPAN6 (7105)" -> "TSPAN6"
def clean_gene(col):
    m = re.match(r"^([A-Za-z0-9\-_]+)\s*\(\d+\)$", str(col))
    return m.group(1) if m else str(col)

expr.columns = ["depmap_id"] + [clean_gene(c) for c in expr.columns[1:]]

# 3) Ensure numeric gene matrix (keep depmap_id as string)
for c in expr.columns[1:]:
    expr[c] = pd.to_numeric(expr[c], errors="coerce")

print("Expression ready:", expr.shape)
print("Example depmap_id:", expr["depmap_id"].iloc[0])
print("Example genes:", expr.columns[1:6].tolist())

Expression ready: (1406, 19222)
Example depmap_id: ACH-001113
Example genes: ['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112']


2) Prepare PRISM target table (IC50/AUC per drug × cell line)

In [ ]:
# Keep only what we need from PRISM
train = prism[[cell_col, drug_col, ic50_uM_col, auc_col]].copy()
train.columns = ["depmap_id", "drug", "ic50_uM", "auc"]

# Clean types
train["depmap_id"] = train["depmap_id"].astype(str).str.strip()
train["drug"] = train["drug"].astype(str).str.strip()

train["ic50_uM"] = pd.to_numeric(train["ic50_uM"], errors="coerce")
train["auc"] = pd.to_numeric(train["auc"], errors="coerce")

# Convert IC50 (uM) -> ln(IC50 in molar)  [optional but recommended]
# molar = uM * 1e-6
train["ln_ic50_M"] = np.log(train["ic50_uM"] * 1e-6)

# Drop unusable rows
train = train.replace([np.inf, -np.inf], np.nan)
train_clean = train.dropna(subset=["depmap_id", "drug", "ln_ic50_M"])

print("PRISM clean:", train_clean.shape)
print(train_clean.head(3))


/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


PRISM clean: (361878, 5)
      depmap_id         drug    ic50_uM       auc  ln_ic50_M
533  ACH-000569   epinastine  32.524180  0.979333 -10.333527
820  ACH-000879  floxuridine   0.501776  0.726373 -14.505112
821  ACH-000320  floxuridine   0.007879  0.336463 -18.659009


3) Merge expression features with PRISM targets

In [ ]:
# Merge: each row becomes (cell line + drug) with gene features
train_full = train_clean.merge(expr, on="depmap_id", how="inner")


NameError: name 'train_clean' is not defined

In [ ]:

print("Merged shape:", train_full.shape)
print("Example columns:", train_full.columns[:10].tolist())


In [ ]:
# 4) Build training matrix X and target y
#Because you can’t train anything until you define X (genes) and y (IC50) and remove missingness.


# Identify gene columns (everything except metadata)
meta_cols = {"depmap_id", "drug", "ic50_uM", "auc", "ln_ic50_M"}
gene_cols = [c for c in train_full.columns if c not in meta_cols]

# X = gene expression, y = ln(IC50 in molar)
X = train_full[gene_cols].apply(pd.to_numeric, errors="coerce")
y = pd.to_numeric(train_full["ln_ic50_M"], errors="coerce")

# Optional: drop genes that are mostly missing
min_non_nan = 0.95
keep_genes = (X.notna().mean(axis=0) >= min_non_nan)
X = X.loc[:, keep_genes]

# Drop rows with too many missing gene values
row_keep = (X.notna().mean(axis=1) >= min_non_nan) & y.notna()
X = X.loc[row_keep].fillna(X.mean())
y = y.loc[row_keep]

# Keep metadata aligned
drug_series = train_full.loc[row_keep, "drug"].astype(str).values
depmap_series = train_full.loc[row_keep, "depmap_id"].astype(str).values

print("Final X shape:", X.shape)
print("Final y shape:", y.shape)
print("Example drug:", drug_series[0])


You must convert that patient sample into the SAME feature space as PRISM/DepMap

Your PRISM model uses DepMap expression genes (columns like TSPAN6, TNMD, …).
Your patient sample uses genes from your RA dataset (symbols like LBP, SEMA3A, …).

So the next step is to build:

✅ patient_x = one-row vector aligned to X.columns (the gene columns you trained with)

Cell A — Build the patient vector aligned to DepMap genes

In [ ]:
# =========================
# Build patient vector aligned to DepMap feature genes (X.columns)
# =========================

# patient_vec: pandas Series indexed by gene symbol, values are expression for selected GSM sample
# mat: your cohort matrix with genes as index and samples as columns (already built)

# Ensure patient gene symbols match DepMap style (uppercase, remove .1 etc)
patient_vec_clean = patient_vec.copy()
patient_vec_clean.index = (
    patient_vec_clean.index.astype(str)
    .str.upper()
    .str.replace(r"\.\d+$", "", regex=True)
    .str.strip()
)

# DepMap feature genes are the columns of X (already cleaned to symbols like TSPAN6)
depmap_genes = pd.Index([g.upper() for g in X.columns])

# Align: keep only genes common between patient and DepMap features
common_genes = depmap_genes.intersection(patient_vec_clean.index)

print("Patient genes:", len(patient_vec_clean))
print("DepMap genes in model:", len(depmap_genes))
print("Common genes:", len(common_genes))

# Build patient vector in DepMap feature space (missing genes -> 0)
patient_x = pd.Series(0.0, index=depmap_genes)
patient_x.loc[common_genes] = patient_vec_clean.loc[common_genes].astype(float).values

# Convert to (1, n_features)
patient_x_arr = patient_x.values.reshape(1, -1)
print("patient_x_arr shape:", patient_x_arr.shape)


In [ ]:
from collections import Counter

# See which drugs have enough samples
drug_counts = Counter(drug_series)
top_drugs = [d for d, n in drug_counts.most_common(30) if n >= 200]

print("Top usable drugs:", len(top_drugs))
print(top_drugs[:10])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import numpy as np

# Pick one drug to test
DRUG_TO_TEST = top_drugs[0]

mask = (drug_series == DRUG_TO_TEST)
X_d = X.loc[mask]
y_d = y.loc[mask]

X_train, X_test, y_train, y_test = train_test_split(
    X_d, y_d, test_size=0.2, random_state=42
)

model = XGBRegressor(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.6,
    reg_lambda=1.0,
    random_state=42
)

model.fit(X_train, y_train)
pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, pred))
r2 = r2_score(y_test, pred)

print("Drug:", DRUG_TO_TEST)
print("Samples:", X_d.shape[0])
print("RMSE:", rmse)
print("R2:", r2)


Cell B — Predict IC50 for ONE drug (the same one you trained)

This gives you your first “digital twin” output:

For this drug, predicted sensitivity for this patient state.

In [ ]:
# =========================
# Predict IC50 for the selected patient sample for this drug
# =========================
pred_ln_ic50 = float(model.predict(patient_x_arr)[0])

# Convert back to IC50 in microM (reverse ln(ic50_uM*1e-6))
pred_ic50_M = np.exp(pred_ln_ic50)
pred_ic50_uM = pred_ic50_M * 1e6

print("Patient sample:", patient_sample)
print("Drug:", DRUG_TO_TEST)
print("Predicted ln(IC50_M):", pred_ln_ic50)
print("Predicted IC50 (uM):", pred_ic50_uM)


Cell C — Predict across many drugs (rank safest/most effective candidates)

This is the real useful output: rank drugs by predicted IC50.

In [ ]:
# =========================
# Train-per-drug quick ranking (MVP-friendly)
# =========================
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import numpy as np
import pandas as pd

results = []

# Choose drugs with enough training samples
from collections import Counter
drug_counts = Counter(drug_series)
usable_drugs = [d for d, n in drug_counts.items() if n >= 300]  # adjust threshold if needed

print("Usable drugs:", len(usable_drugs))

for drug in usable_drugs[:30]:  # start with first 30 to keep runtime reasonable
    mask = (drug_series == drug)
    X_d = X.loc[mask]
    y_d = y.loc[mask]

    if len(y_d) < 300:
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        X_d, y_d, test_size=0.2, random_state=42
    )

    m = XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.6,
        reg_lambda=1.0,
        random_state=42
    )
    m.fit(X_train, y_train)

    # evaluate quickly
    pred_test = m.predict(X_test)
    rmse = float(np.sqrt(mean_squared_error(y_test, pred_test)))

    # predict patient
    pred_ln = float(m.predict(patient_x_arr)[0])
    pred_uM = float(np.exp(pred_ln) * 1e6)

    results.append({
        "drug": drug,
        "n_samples": int(len(y_d)),
        "rmse": rmse,
        "pred_ic50_uM_for_patient": pred_uM
    })

rank_df = pd.DataFrame(results).sort_values("pred_ic50_uM_for_patient", ascending=True)
rank_df.head(15)
